In [82]:
import pandas as pd
import jieba
from gensim.models import word2vec
import numpy as np
#导入Keras中的卷积工具
from keras.models import Sequential  #基础的Keras神经网络模型
from keras.layers import Dense, Dropout, Activation #模型中常用的层对象
from keras.layers import Conv1D,GlobalMaxPooling1D #卷积层和池化


df = pd.read_csv('./data.csv')
df=df[['text','location', 'country']]

#print("在 text 列中总共有 %d 个空值." % df['text'].isnull().sum())
#print("在 location 列中总共有 %d 个空值." % df['location'].isnull().sum())
df[df.isnull().values==True]
df = df[pd.notnull(df['text'])]
#print(df)

df.sample(2)
texts = [[word for word in jieba.cut(document)] for document in df['text']]
df['country_id'] = df['country'].factorize()[0]
df['location_id'] = df['location'].factorize()[0]
cat_id_df = df[['country', 'country_id']].drop_duplicates().sort_values('country_id').reset_index(drop=True)
cat_id_df = df[['location', 'location_id']].drop_duplicates().sort_values('location_id').reset_index(drop=True)

#训练
# word_model = word2vec.Word2Vec(texts,vector_size=100,min_count=3,window=6,hs=1)
word_model = word2vec.Word2Vec(texts, hs=1,min_count=1,window=6,vector_size=2)
word_model.save('word_model')  # 保存模型
# word_model = word2vec.Word2Vec.load('word_model')  # 加载模型
# print(model.wv.similarity('南海', '日本海'))
# print(model.wv['南海'])
# for val in model.wv.similar_by_word("南海", topn=10):
#     print(val[0], val[1])
# for token in texts:
#     print(model.wv[token])


print(texts)
# 直接词向量相加求平均
def fea_sentence(list_w):
    n0 = np.array([0. for i in range(2)], dtype=np.float32)
    for i in list_w:
        n0 += i
    fe = n0 / len(list_w)
    fe = fe.tolist()
    return fe

# 向量表
vectorized_data=[]
for text in texts:
    sample_vecs = []
    for token in text:
        try:
            sample_vecs.append(word_model.wv[token])
        except KeyError:
            pass
    vectorized_data.append(fea_sentence(sample_vecs))
x_train=vectorized_data
y_train=df["country_id"]
print('x_train', x_train)
print('y_train', y_train)
print('x_train', np.shape(x_train))
print('y_train', np.shape(y_train))


# CNN 模型
maxlen=400
batch_size=32 #在后向传播误差和更新权重前，向网络传入的样本数量
embedding_dims=2 #传入卷积神经网络中词条向量的长度
filters=250 #要训练的卷积核的数量
kernel_size=3 #卷积核大小：每个卷积核将是一个矩阵：embedding_dims*kernel_size
hidden_dims=250 #在普通的前馈网络中传播链端点的神经元的数量
epochs=1 #整个训练集在网络中的传入次数
#定义模型
model = Sequential()
model.add(Conv1D(
    filters,
    kernel_size,
    padding='valid',
    activation='relu',
    strides=1,
    input_shape=(maxlen,embedding_dims)
))
#池化
model.add(GlobalMaxPooling1D())
#带dropout的全连接层
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
#漏斗funnel
model.add(Dense(1))
model.add(Activation('sigmoid'))
#编译CNN
model.compile(loss='sparse_categorical_crossentropy', #多分类的损失函数
              optimizer='adam',
              metrics=['accuracy']
              )
#categorical变量的输出层
model.add(Dense(10))
model.add(Activation('sigmoid'))
#训练CNN
model.fit(
 x_train,
 y_train,
 batch_size=batch_size,
 epochs=epochs,
 #  validation_split = 0.2
 # 训练集的20%用作验证集
)

# 保存模型
model_structure=model.to_json()
with open("cnn_model.json","w") as json_file:
    json_file.write(model_structure)
model.save_weights('model_CNN_text.h5')


[['美日于', '2023', '年', '9', '月', '10', '日', '在', '南海', '举行', '联合', '军演'], ['美韩', '在', '日本海', '举行', '联合', '军演']]


ValueError: operands could not be broadcast together with shapes (10,) (2,) (10,) 

In [39]:
def predict(text):
    df = pd.read_csv('./data.csv')
    df=df[['text','location', 'country']]

    df[df.isnull().values==True]
    df = df[pd.notnull(df['text'])]


    df['text_id'] = df['text'].factorize()[0]
    cat_id_df = df[['text', 'text_id']].drop_duplicates().sort_values('text_id').reset_index(drop=True)
    cat_to_id = dict(cat_id_df.values)
    id_to_cat = dict(cat_id_df[['text_id', 'text']].values)


    texts = [[word for word in jieba.cut(document)]for document in df['text']]
    for txt in texts:
        sample_vecs = []
        for token in txt:
            try:
                sample_vecs.append(model.wv[token])
            except KeyError:
                pass
    padded = np.reshape(sample_vecs,maxlen=maxlen)
    pred = model.predict(padded)
    cat_id= pred.argmax(axis=1)[0]
    return cat_id_df[cat_id_df.cat_id==cat_id]['cat'].values[0]

predict("衣服真的很大很大，但是颜色好看，字母也很有质感，喜欢喜欢")
predict("美韩在日本海举行联合军演")

NameError: name 'np' is not defined